In [19]:
import pandas as pd
import math
import os

In [20]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

def configure_driver():
    #Configurations

     # Đường dẫn đến WebDriver
    webdriver_path = "../driver/chromedriver.exe"
    chrome_options = Options()

    # Tắt thông báo Chrome - chặn pop-up
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--start-maximized")
    # chrome_options.add_argument("--disable-notifications") # disable notifications
    # chrome_options.add_argument("--start-maximized") # disable infobars
    chrome_options. add_argument('--blink-settings=imagesEnabled=false') # disable images
    #chrome_options.add_argument('--window-size=800,600')
    
    # Đặt vị trí cửa sổ ở ngoài màn hình chính
    #chrome_options.add_argument('--window-position=2000,0')
    #chrome_options.add_extension("F:\GlobalTech\Stands AdBlocker - Chrome Web Store 2.1.24.0.crx")

    # Sử dụng profile đã có sẵn: 
    # để lưu trạng thái đăng nhập + dùng tool chặn quảng cáo set up sẵn trên Chrome
    # user_profile = "C:\\Users\\21521\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 5"
    # chrome_options.add_argument("user-data-dir=" + user_profile)

    # (Tùy chọn) Chạy Chrome ở chế độ không hiển thị giao diện (chạy ngầm)
    # chrome_options.add_argument("--headless")

    # Khởi tạo dịch vụ và trình duyệt
    service = Service(webdriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    return driver

In [21]:
# Hàm thu thập liên kết công việc từ một trang
def get_one_page():
    # Tìm tất cả thẻ <a> chứa liên kết công việc
    elements = driver.find_elements(By.CSS_SELECTOR, "h2 a[class='job_link']")
    # Lấy thuộc tính 'href' của từng phần tử
    page_urls = [element.get_attribute("href") for element in elements]
    return page_urls


# Hàm duyệt qua nhiều trang và thu thập liên kết
def get_all_pages(base_url, max_pages):
    all_urls = []

    for page in range(1, max_pages + 1):
        url = f'{base_url}{page}'
        driver.get(url)
        # Thu thập liên kết trên trang hiện tại
        links = get_one_page()
        all_urls.extend(links)
        
    return all_urls

In [22]:
def get_job_detail(url, driver):
    driver.get(url)
    print(f'Đang xử lý: {url}')
    job_detail = {}

    # Xử lý cho tab chi tiết công việc

    # Lấy tên công việc
    try:
        job_name_element = driver.find_element(By.CSS_SELECTOR, "h1[class='title']")
    except Exception:
        try:
            job_name_element = driver.find_element(By.CSS_SELECTOR, "div[class='title'] h2")
        except Exception:
            job_name_element = None

    if job_name_element:
        job_name = job_name_element.text.strip()
    else:
        job_name = None
    job_detail['Tên công việc'] = job_name

    job_locations_elements = driver.find_elements(By.CSS_SELECTOR, "div[class='map'] p a")
    job_locations = [element.text for element in job_locations_elements]
    job_detail['Địa điểm'] = job_locations

    # Lấy địa chỉ 
    job_elements = driver.find_elements(By.CLASS_NAME, 'has-background')
    for element in job_elements:
        li_elements = element.find_elements(By.CSS_SELECTOR, "li")
        for li in li_elements:
            # Lấy tiêu đề từ thẻ <strong>
            label = li.find_element(By.CSS_SELECTOR, "strong").text
            # Lấy giá trị từ thẻ <p>
            value = li.find_element(By.CSS_SELECTOR, "p").text.strip()
            job_detail[label] = value

    # Chuyển tab (nếu có)
    try:
        # overview_tab = WebDriverWait(driver, 5).until(
        #         EC.element_to_be_clickable((By.XPATH, "//a[@title='Tổng quan công ty']"))
        #     )
        overview_tab = driver.find_element(By.XPATH, "//a[@title='Tổng quan công ty']")
        overview_tab.click()

        # Xử lý tab thông tin công ty
        company_name_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a[class='name']"))
        )
        company_name = company_name_element.text.strip()
        job_detail['Tên công ty'] = company_name

        # Tìm thẻ cha chứa "Địa điểm"
        company_content_element = driver.find_element(By.XPATH, ".//div[@class='company-info']//strong[text()='Địa điểm']/..")

        # Lấy toàn bộ nội dung văn bản trong phần tử cha
        company_content = company_content_element.text

        # Tìm phần địa chỉ công ty
        if "Địa điểm" in company_content:
            # Tách lấy phần sau "Địa điểm"
            address_part = company_content.split("Địa điểm", 1)[1]
            
            # Lấy phần trước "Thông tin công ty"
            company_address = address_part.split("Thông tin công ty", 1)[0].strip()
            
            job_detail['Địa chỉ công ty'] = company_address

        company_li = driver.find_elements(By.CSS_SELECTOR, "div[class='company-info'] ul li")
        for li in company_li:
            text = li.text
            if "Người liên hệ" in text:
                job_detail['Người liên hệ'] = text.split(":")[1].strip()
            elif "Qui mô công ty" in text:
                job_detail['Qui mô công ty'] = text.split(":")[1].strip()
            elif "Loại hình hoạt động" in text:
                job_detail['Loại hình hoạt động'] = text.split(":")[1].strip()
    except Exception as e:
        print("Không tìm thấy tab Tổng quan công ty hoặc thông tin công ty.")
        # Thêm giá trị mặc định nếu không tìm thấy tab
        job_detail['Tên công ty'] = None
        job_detail['Địa chỉ công ty'] = None
        job_detail['Người liên hệ'] = None
        job_detail['Qui mô công ty'] = None
        job_detail['Loại hình hoạt động'] = None

    return job_detail


In [23]:
def save_links_to_files(links, num_files):
    chunk_size = math.ceil(len(links) / num_files)
    for i in range(num_files):
        file_links = links[i * chunk_size:(i + 1) * chunk_size]
        file_name = f"../dataset/job_links_part_{i + 1}.txt"
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write('\n'.join(file_links))
        print(f"Lưu {len(file_links)} link vào {file_name}")

In [24]:
# job_urls = get_all_pages(base_url, max_pages=500)
# save_links_to_files(job_urls, 4)

In [25]:
def read_links_from_file(file_name):
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            links = file.read().splitlines()  # Đọc từng dòng và loại bỏ ký tự xuống dòng
        return links
    except FileNotFoundError:
        print(f"File '{file_name}' không tồn tại.")
        return []


In [26]:
driver = configure_driver()
base_url = "https://careerviet.vn/viec-lam-noi-bat-trong-tuan-l8a30r50p"

In [27]:
# Đọc từ một file cụ thể
input_file = "../dataset/job_links_part_3.txt"  # Thay tên file bạn muốn đọc
job_urls = read_links_from_file(input_file)

# Kiểm tra danh sách URL
print(f"Số URL trong file '{input_file}':", len(job_urls))
print("Ví dụ URL:", job_urls[:5])  # Hiển thị 5 URL đầu tiên

Số URL trong file '../dataset/job_links_part_3.txt': 6129
Ví dụ URL: ['https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh.35C27D58.html', 'https://careerviet.vn/vi/tim-viec-lam/truong-phong-ky-thuat-san-xuat.35C27775.html', 'https://careerviet.vn/vi/tim-viec-lam/trinh-duoc-vien-thuc-pham-chuc-nang-cham-soc-suc-khoe-an-giang.35C276BC.html', 'https://careerviet.vn/vi/tim-viec-lam/ky-su-qs.35C27652.html', 'https://careerviet.vn/vi/tim-viec-lam/ky-su-qa-qc.35C27616.html']


In [28]:
job_list = []
for url in job_urls:
    try:
        job_detail = get_job_detail(url, driver)
        job_list.append(job_detail)
    except:
        continue


Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh.35C27D58.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/truong-phong-ky-thuat-san-xuat.35C27775.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/trinh-duoc-vien-thuc-pham-chuc-nang-cham-soc-suc-khoe-an-giang.35C276BC.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/ky-su-qs.35C27652.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/ky-su-qa-qc.35C27616.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/ky-su-shopdrawing.35C27611.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-hanh-chinh.35C27332.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/qa-staff.35C26FF1.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/operation-intern.35C2701B.html
Không tìm thấy tab Tổng quan công ty hoặc thông tin công ty.
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/technical-department-supervisor.35C26C23.html
Đang xử lý: https://careerviet.vn/vi/tim-viec-lam/trinh-duoc-vien-

In [29]:
df = pd.DataFrame(job_list)
output_file = "../dataset/jobs_part_3.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print("Dữ liệu đã được lưu vào jobs.csv")

Dữ liệu đã được lưu vào jobs.csv
